In [ ]:
!pip install transformers

In [ ]:
!pip install sentence_transformers

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
# 임시 코드 입니다

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Leo97/KoELECTRA-small-v3-modu-ner")
model = AutoModelForTokenClassification.from_pretrained("Leo97/KoELECTRA-small-v3-modu-ner")
ner = pipeline("ner", model=model, tokenizer=tokenizer)

In [ ]:
example = '한글 데이터의 개체명 인식 동작을 확인합니다.'
print(ner(example))

[{'entity': 'B-CV', 'score': 0.60997397, 'index': 1, 'word': '한글', 'start': 0, 'end': 2}]


In [ ]:
def return_words(text):
  '''
  input : text
  return : text의 키워드 후보와 index가 있는 dict
  '''
  ner_results = ner(text)

  dic = {'word':[], 'score':[], 'entity':[], 'index':[]}
  temp_str = ''
  temp_score = []
  temp_entity = []
  temp_index = []
  for result in ner_results:
    if result['entity'][0] == 'B':
      if temp_str != '':
        dic['word'].append(temp_str)
        dic['score'].append(temp_entity)
        dic['entity'].append(temp_score)
        dic['index'].append(temp_index)
        temp_str = ''
        temp_score = []
        temp_entity = []
        temp_index = []

      temp_str += result['word']

    elif result['entity'][0] == 'I':
      if result['word'][0] == '#':
        temp_str += result['word'][2:]
      else:
        temp_str += ' ' + result['word']
    temp_score.append(result['score'])
    temp_entity.append(result['entity'])
    temp_index.append((result['start'], result['end']))

  if temp_str != '':
    dic['word'].append(temp_str)
    dic['score'].append(temp_entity)
    dic['entity'].append(temp_score)
    dic['index'].append(temp_index)

  return dic

In [ ]:
text = '‘챗 GPT’ 등의 출현으로 미국 반도체 업체 엔비디아 주가가 연일 고공행진을 이어가자 차기 수혜주에 관심이 쏠리고 있다.'
result = return_words(text)
words = result['word']
print(words)
result

['챗 GPT', '미국', '반도체', '엔비디아']


{'word': ['챗 GPT', '미국', '반도체', '엔비디아'],
 'score': [['B-TM', 'I-TM', 'I-TM'],
  ['B-LC'],
  ['B-TM'],
  ['B-OG', 'I-OG', 'I-OG']],
 'entity': [[0.34701303, 0.33055353, 0.37826726],
  [0.9544251],
  [0.84381807],
  [0.9506789, 0.94155383, 0.94034964]],
 'index': [[(1, 2), (3, 5), (5, 6)],
  [(16, 18)],
  [(19, 22)],
  [(26, 27), (27, 28), (28, 30)]]}

### csv 파일로 확인

In [ ]:
df = pd.read_csv('056730.csv')

In [ ]:
df.head()

,company,title,link,writer,date,content
0,CNT85,"CNT85, 156.5억 규모 공사 계약 체결",https://finance.naver.com/item/news_read.naver...,이데일리,2023.07.03 13:46,[이데일리 원다연 기자] CNT85(056730)는 테크로스워터앤에너지와 156억 ...
1,CNT85,"CNT85, 28억 규모 RTO 설치공사 공급계약",https://finance.naver.com/item/news_read.naver...,이데일리,2023.06.16 14:30,[이데일리 김보겸 기자] CNT85(056730)는 28억원 규모의 당진공장 축열식...
2,CNT85,"CNT85, 4.6억 규모 전환청구권 행사",https://finance.naver.com/item/news_read.naver...,이데일리,2023.05.26 16:22,[이데일리 김응태 기자] CNT85(056730)는 4억6000만원 규모의 23회차...
3,CNT85,"CNT85, 1분기 영업익 2.3억원…흑자전환 성공",https://finance.naver.com/item/news_read.naver...,머니투데이,2023.05.12 14:42,"플랜트, 필터·여과수처리 사업을 운영하는 CNT85가 1분기 영업이익 2억3000만..."
4,CNT85,"CNT85, 7억 규모 전환청구권 행사",https://finance.naver.com/item/news_read.naver...,이데일리,2023.04.28 17:25,[이데일리 이정현 기자] CNT85(056730)는 7억 원 규모의 23회차 무기명...


In [ ]:
text = df.loc[0]['content']

In [ ]:
title = df.loc[0]['title']

In [ ]:
text

'[이데일리 원다연 기자] CNT85(056730)는 테크로스워터앤에너지와 156억 5000만원 규모의 기흥 SDR 그린센터 오수처리시설 기계배관 공사 계약을 체결했다고 3일 공시했다.이는 최근 매출액 대비 48.78%에 해당한다. 계약 기간은 지난 30일부터 내년 12월 31일까지다.'

In [ ]:
result = return_words(text)
words = result['word']
words

['일리',
 '원다연',
 '기자',
 'CNT85',
 '0567',
 '##30',
 '테크로스워터앤에너지',
 '156억 5000',
 '기흥',
 'SDR 그린센터',
 '3',
 '##일',
 '48 . 78 %']

In [ ]:
'''
  todo : 현재 입력 데이터의 길이가 길면 출력이 이상해 지는 문제가 있음
  아래 예시 확인
'''

In [ ]:
result = return_words(text[:50])
words = result['word']
words

['원다연', '기자', 'CNT85', '056730', '테크로스워터앤에너지', '156억 5000']

In [ ]:
result = return_words(text[:101])
words = result['word']
words

['원다연',
 '기자',
 'CNT85',
 '056730',
 '테크로스워터앤에너지',
 '156억 5000',
 '기흥',
 'SDR 그린센터',
 '3일']

In [ ]:
result = return_words(text[:130])
words = result['word']
words

['일리',
 '원다연',
 '기자',
 'CNT85',
 '056730',
 '테크로스워터앤에너지',
 '156억 5000',
 '기흥',
 'SDR 그린센터',
 '3일',
 '48 . 78 %']

In [ ]:
result = return_words(text)
words = result['word']
words

['일리',
 '원다연',
 '기자',
 'CNT85',
 '0567',
 '##30',
 '테크로스워터앤에너지',
 '156억 5000',
 '기흥',
 'SDR 그린센터',
 '3',
 '##일',
 '48 . 78 %']

In [ ]:
result

{'word': ['일리',
  '원다연',
  '기자',
  'CNT85',
  '0567',
  '##30',
  '테크로스워터앤에너지',
  '156억 5000',
  '기흥',
  'SDR 그린센터',
  '3',
  '##일',
  '48 . 78 %'],
 'score': [['I-OG'],
  ['B-OG', 'I-OG', 'I-PS'],
  ['B-CV'],
  ['B-AF', 'I-AF', 'I-AF'],
  ['B-QT', 'I-QT'],
  ['B-QT'],
  ['B-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG'],
  ['B-QT', 'I-QT', 'I-QT'],
  ['B-LC'],
  ['B-AF', 'I-AF', 'I-AF', 'I-AF'],
  ['B-DT'],
  ['B-DT'],
  ['B-QT', 'I-QT', 'I-QT', 'I-QT']],
 'entity': [[0.49553466],
  [0.5833473, 0.45962462, 0.5877794],
  [0.61917484],
  [0.41322687, 0.45933244, 0.35766423],
  [0.82425725, 0.73227006],
  [0.80768573],
  [0.86274034,
   0.8953028,
   0.89671725,
   0.92597497,
   0.9127999,
   0.9244243,
   0.9098607],
  [0.63297695, 0.64723974, 0.66578895],
  [0.7482972],
  [0.33515036, 0.42516705, 0.46068886, 0.416388],
  [0.7863837],
  [0.77908856],
  [0.7852038, 0.78137255, 0.8358253, 0.80113524]],
 'index': [[(3, 5)],
  [(6, 7), (7, 8), (8, 9)],
  [(10, 12)],
  [(14, 16), (16,

In [ ]:
result = return_words(text[:130])
words = result['word']
words

['일리',
 '원다연',
 '기자',
 'CNT85',
 '056730',
 '테크로스워터앤에너지',
 '156억 5000',
 '기흥',
 'SDR 그린센터',
 '3일',
 '48 . 78 %']

### 불용어 제거

In [ ]:
#stopword 목록 다운
!git clone https://gist.github.com/d10b18575a73778da4bc83853385465c.git stopwords

Cloning into 'stopwords'...
remote: Enumerating objects: 3, done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (3/3), 2.75 KiB | 2.75 MiB/s, done.


In [ ]:
'''
  todo : 불용어 추가
'''
def load_stopwords(custom_stopwords = None):
  words = []

  if custom_stopwords:
    words.extend(custom_stopwords)

  file_path = "./stopwords/stopwords.txt"
  with open(file_path, 'r') as file:
      words.extend(file.read().split('\n'))

  # file_path = "./stopword.txt"
  # with open(file_path, 'r') as file:
  #     words.extend(file.read().split('\n'))

  # file_path = "./stopword2.txt"
  # with open(file_path, 'r') as file:
  #     words.extend(file.read().split('\n')[1:])

  return list(set(words))

In [ ]:
# custom = []
custom = ['기자','3일']
stopwords = load_stopwords(custom)

In [ ]:
print(len(stopwords))

597


In [ ]:
result

{'word': ['일리',
  '원다연',
  '기자',
  'CNT85',
  '056730',
  '테크로스워터앤에너지',
  '156억 5000',
  '기흥',
  'SDR 그린센터',
  '3일',
  '48 . 78 %'],
 'score': [['I-OG'],
  ['B-OG', 'I-PS', 'I-PS'],
  ['B-CV'],
  ['B-AF', 'I-AF', 'I-AF'],
  ['B-QT', 'I-QT', 'I-QT'],
  ['B-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG'],
  ['B-QT', 'I-QT', 'I-QT'],
  ['B-LC'],
  ['B-AF', 'I-AF', 'I-AF', 'I-AF'],
  ['B-DT', 'I-DT'],
  ['B-QT', 'I-QT', 'I-QT', 'I-QT']],
 'entity': [[0.4308679],
  [0.5620235, 0.4328442, 0.6262465],
  [0.6986592],
  [0.42377126, 0.45967755, 0.36286187],
  [0.9009548, 0.8883027, 0.940171],
  [0.87209386,
   0.89965653,
   0.89785665,
   0.9268155,
   0.9119154,
   0.9234293,
   0.9095367],
  [0.64638066, 0.6557225, 0.6707427],
  [0.7661712],
  [0.37843058, 0.4347338, 0.47299442, 0.4301917],
  [0.8726086, 0.946883],
  [0.83745456, 0.8217816, 0.870865, 0.8377137]],
 'index': [[(3, 5)],
  [(6, 7), (7, 8), (8, 9)],
  [(10, 12)],
  [(14, 16), (16, 17), (17, 19)],
  [(20, 22), (22, 24), (24, 

In [ ]:
remove_idxs = [i for i, word in enumerate(result['word']) if word in stopwords]
remove_idxs

[2, 9]

In [ ]:
for idx in sorted(remove_idxs, reverse=True):
    del result['word'][idx]
    del result['score'][idx]
    del result['entity'][idx]
    del result['index'][idx]

In [ ]:
result

{'word': ['일리',
  '원다연',
  'CNT85',
  '056730',
  '테크로스워터앤에너지',
  '156억 5000',
  '기흥',
  'SDR 그린센터',
  '48 . 78 %'],
 'score': [['I-OG'],
  ['B-OG', 'I-PS', 'I-PS'],
  ['B-AF', 'I-AF', 'I-AF'],
  ['B-QT', 'I-QT', 'I-QT'],
  ['B-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG', 'I-OG'],
  ['B-QT', 'I-QT', 'I-QT'],
  ['B-LC'],
  ['B-AF', 'I-AF', 'I-AF', 'I-AF'],
  ['B-QT', 'I-QT', 'I-QT', 'I-QT']],
 'entity': [[0.4308679],
  [0.5620235, 0.4328442, 0.6262465],
  [0.42377126, 0.45967755, 0.36286187],
  [0.9009548, 0.8883027, 0.940171],
  [0.87209386,
   0.89965653,
   0.89785665,
   0.9268155,
   0.9119154,
   0.9234293,
   0.9095367],
  [0.64638066, 0.6557225, 0.6707427],
  [0.7661712],
  [0.37843058, 0.4347338, 0.47299442, 0.4301917],
  [0.83745456, 0.8217816, 0.870865, 0.8377137]],
 'index': [[(3, 5)],
  [(6, 7), (7, 8), (8, 9)],
  [(14, 16), (16, 17), (17, 19)],
  [(20, 22), (22, 24), (24, 26)],
  [(29, 31), (31, 33), (33, 34), (34, 35), (35, 36), (36, 37), (37, 39)],
  [(41, 44), (44, 45),

In [ ]:
words = result['word']
words

['일리',
 '원다연',
 'CNT85',
 '056730',
 '테크로스워터앤에너지',
 '156억 5000',
 '기흥',
 'SDR 그린센터',
 '48 . 78 %']

### 벡터화

In [ ]:
'''
  참고 사이트 :
  https://devocean.sk.com/blog/techBoardDetail.do?ID=164033
'''

In [ ]:
'''
  todo : sota 모델 찾기
'''
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [ ]:
print(title)
print(text)
print(words)

CNT85, 156.5억 규모 공사 계약 체결
[이데일리 원다연 기자] CNT85(056730)는 테크로스워터앤에너지와 156억 5000만원 규모의 기흥 SDR 그린센터 오수처리시설 기계배관 공사 계약을 체결했다고 3일 공시했다.이는 최근 매출액 대비 48.78%에 해당한다. 계약 기간은 지난 30일부터 내년 12월 31일까지다.
['일리', '원다연', 'CNT85', '056730', '테크로스워터앤에너지', '156억 5000', '기흥', 'SDR 그린센터', '48 . 78 %']


In [ ]:
# title 벡터화
title_emb = model.encode(title)
# text 벡터화
doc_emb = model.encode(text)
# words 백터화
word_emb = model.encode(words)
# print(doc_emb.shape)
# print(word_emb.shape)

In [ ]:
# 코사인 유사도 측정
title_word_dists = cosine_distances([title_emb], word_emb).flatten()
text_word_dists = cosine_distances([doc_emb], word_emb).flatten()
print('title과 text의 cosine distance: ',cosine_distances([doc_emb], [title_emb]).flatten())

title과 text의 cosine distance:  [0.36870557]


In [ ]:
text_word_dists

array([0.87537664, 0.82498133, 0.5951872 , 0.6734893 , 0.49724144,
       0.6993714 , 0.6877056 , 0.6774784 , 0.71606404], dtype=float32)

In [ ]:
print('title과 words의 cosine distance')
for i in np.argsort(title_word_dists):
    print(words[i],title_word_dists[i])

title과 words의 cosine distance
CNT85 0.36422163
156억 5000 0.5453429
테크로스워터앤에너지 0.58205175
056730 0.6712269
SDR 그린센터 0.7803867
원다연 0.8208597
기흥 0.82504976
일리 0.8513015
48 . 78 % 0.97520405


In [ ]:
print('text과 words의 cosine distance')
for i in np.argsort(text_word_dists):
    print(words[i],text_word_dists[i])

text과 words의 cosine distance
테크로스워터앤에너지 0.49724144
CNT85 0.5951872
056730 0.6734893
SDR 그린센터 0.6774784
기흥 0.6877056
156억 5000 0.6993714
48 . 78 % 0.71606404
원다연 0.82498133
일리 0.87537664


### Score 계산

In [ ]:
# score 구하기 (임시)
score = 0.6 * (1 - title_word_dists) + 0.4 * (1 - text_word_dists)
score

array([0.13906845, 0.17749166, 0.5433922 , 0.32786816, 0.4518724 ,
       0.3930457 , 0.22988792, 0.26077664, 0.12845196], dtype=float32)

In [ ]:
print(list(score))
np.argsort(score)

[0.13906845, 0.17749166, 0.5433922, 0.32786816, 0.4518724, 0.3930457, 0.22988792, 0.26077664, 0.12845196]


array([8, 0, 1, 6, 7, 3, 5, 4, 2])

In [ ]:
top_k = 3
top_words = []
for i in np.argsort(-score)[:top_k]:
    top_words.append()
    print(i, words[i], 'score :',score[i], 'title score :', 1 - title_word_dists[i], 'text score :', 1 - text_word_dists[i])

2 CNT85 score : 0.5433922 title score : 0.6357783675193787 text score : 0.4048128128051758
4 테크로스워터앤에너지 score : 0.4518724 title score : 0.41794824600219727 text score : 0.5027585625648499
5 156억 5000 score : 0.3930457 title score : 0.45465707778930664 text score : 0.3006286025047302


In [ ]:
print(title)
text

CNT85, 156.5억 규모 공사 계약 체결


'[이데일리 원다연 기자] CNT85(056730)는 테크로스워터앤에너지와 156억 5000만원 규모의 기흥 SDR 그린센터 오수처리시설 기계배관 공사 계약을 체결했다고 3일 공시했다.이는 최근 매출액 대비 48.78%에 해당한다. 계약 기간은 지난 30일부터 내년 12월 31일까지다.'

In [ ]:
result['word'].index()

In [ ]:
'''
  todo : 유사도 threshold로 word filtering
'''

### 키워드 빈도수 계산

In [ ]:
word_counts = {}
for word in words:
    count = text.count(word)
    word_counts[word] = count

In [ ]:
word_counts

{'챗': 1,
 '등': 1,
 '출현': 1,
 '미국 반도체 업체 엔비디아 주가': 1,
 '연일': 1,
 '행진': 1,
 '어가': 1,
 '차기': 1,
 '혜주': 1,
 '관심': 1}

In [ ]:
'''
  todo :
  1. 뉴스 제목과 word의 유사도 측정
  2. 빈도수, 제목과 유사성, 본문과 유사성 등 을 합쳐 점수 계산
  (option) 기사 간 키워드 클러스터링 해서 filtering
'''

In [ ]:
'''
  todo : option
  급상승 키워드 재순위화
'''

In [ ]:
'''
나중에 참고용
elastric search 도입 참고용
https://www.lesstif.com/dbms/elastic-search-nori-80248960.html
'''